In [12]:
#import modules

import pandas as pd
from PIL import Image
import numpy as np 
from os import walk

def change(string):
    result = ''
    for i in range(len(string)):
        if string[i] == "\\":
            
            result += '/'
        else:
            result += string[i]
    return result[2:] if result[0] == 'C' else result  

def add_path(number_list,before): #absolute path
    for i in range(len(number_list)):
        number_list[i] = before+'/'+str(number_list[i])
    return number_list

In [27]:
#checking the csv 
df = pd.read_csv('/Users/elanu/Desktop/data/Bird images (kaggle)/birds.csv')
df.head(350)

,class id,filepaths,labels,scientific label,data set
0,0,train/ABBOTTS BABBLER/001.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
1,0,train/ABBOTTS BABBLER/002.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
2,0,train/ABBOTTS BABBLER/003.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
3,0,train/ABBOTTS BABBLER/004.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
4,0,train/ABBOTTS BABBLER/005.jpg,ABBOTTS BABBLER,Malacocincla abbotti,train
...,...,...,...,...,...
345,1,train/ABBOTTS BOOBY/180.jpg,ABBOTTS BOOBY,Papasula abbotti,train
346,2,train/ABYSSINIAN GROUND HORNBILL/001.jpg,ABYSSINIAN GROUND HORNBILL,Bucorvus abyssinicus,train
347,2,train/ABYSSINIAN GROUND HORNBILL/002.jpg,ABYSSINIAN GROUND HORNBILL,Bucorvus abyssinicus,train
348,2,train/ABYSSINIAN GROUND HORNBILL/003.jpg,ABYSSINIAN GROUND HORNBILL,Bucorvus abyssinicus,train


In [32]:
#function to make the three lists, train test and valid with filepath and species names

def list_info_images(folder_dir, test_train_or_valid):
    data_list = []
    for (dirpath, dirnames, filenames) in walk(folder_dir): # folder_dir = test, train or valid folder
        dirnames = add_path(dirnames,folder_dir) 
        for folders_insect in dirnames: #for each folder in the list of all subdirectories of database
            folder_name = folders_insect.replace(folder_dir+'/', '') #to get the folder/species name 
            for (dirpath1, dirnames1, filenames1) in walk(folders_insect): # inside one of the folders
                for pictures in filenames1: #loop through pictures of an insect(=folder_insect)  
                    pic_path = dirpath1 + '/' + pictures  #path of each individual picture
                    sig_pic_path = pic_path.replace(folder_dir, test_train_or_valid)  #picture path without redundant main directory
                    info_image = [] #list refreshes each loop to get [(im_size), 'filepath', 'folder', 'img.jpg']  for all outliers
                    info_image.append(sig_pic_path) 
                    info_image.append(folder_name)
                    data_list.append(info_image) #append [(im_size), 'filepath', 'folder', 'img.jpg']  to the outliers list
    return data_list

In [37]:
folder_test = change(r'C:\Users\elanu\Desktop\data\Bird images (kaggle)\test')
folder_train = change(r'C:\Users\elanu\Desktop\data\Bird images (kaggle)\train')
folder_valid = change(r'C:\Users\elanu\Desktop\data\Bird images (kaggle)\valid')

# #paths for gpu server
# folder_test = './test'
# folder_train = './train'
# folder_valid = './valid'

test_list = list_info_images(folder_test, '/test')
train_list = list_info_images(folder_train, '/train')
valid_list = list_info_images(folder_valid, '/valid')

# print(test_list[:5])
# print()
# print(train_list[:5])
# print()
# print(valid_list[:5])

[['/test/ABBOTTS BABBLER/1.jpg', 'ABBOTTS BABBLER'], ['/test/ABBOTTS BABBLER/2.jpg', 'ABBOTTS BABBLER'], ['/test/ABBOTTS BABBLER/3.jpg', 'ABBOTTS BABBLER'], ['/test/ABBOTTS BABBLER/4.jpg', 'ABBOTTS BABBLER'], ['/test/ABBOTTS BABBLER/5.jpg', 'ABBOTTS BABBLER']]

[['/train/ABBOTTS BABBLER/001.jpg', 'ABBOTTS BABBLER'], ['/train/ABBOTTS BABBLER/002.jpg', 'ABBOTTS BABBLER'], ['/train/ABBOTTS BABBLER/003.jpg', 'ABBOTTS BABBLER'], ['/train/ABBOTTS BABBLER/004.jpg', 'ABBOTTS BABBLER'], ['/train/ABBOTTS BABBLER/005.jpg', 'ABBOTTS BABBLER']]

[['/valid/ABBOTTS BABBLER/1.jpg', 'ABBOTTS BABBLER'], ['/valid/ABBOTTS BABBLER/2.jpg', 'ABBOTTS BABBLER'], ['/valid/ABBOTTS BABBLER/3.jpg', 'ABBOTTS BABBLER'], ['/valid/ABBOTTS BABBLER/4.jpg', 'ABBOTTS BABBLER'], ['/valid/ABBOTTS BABBLER/5.jpg', 'ABBOTTS BABBLER']]


In [52]:
#source: https://stackoverflow.com/questions/49649215/pandas-image-to-dataframe/49650363#49650363
#create {table:[height, width, r, g, b]} for all resized images and append them to one list values

values = []
c = 0

for row in test_list:
#     if c == 1:
#         break
    pic_path = folder_test.replace('/test', row[0])
    colourImg = Image.open(pic_path)
    colourPixels = colourImg.convert("RGB")
    colourArray = np.array(colourPixels.getdata()).reshape(colourImg.size + (3,))
    indicesArray = np.moveaxis(np.indices(colourImg.size), 0, 2) #changes (2, 256, 256)(=colourImg.size) to (256, 256, 2)
    allArray = np.dstack((indicesArray, colourArray)).reshape((-1, 5))
    value = [] #[array, 'folder/insectname' , '/folder/img.jpg']
    value.append(allArray)
    value.append(row[1])
    value.append(row[0])
    values.append(value)
#     c += 1

    
#Here we will insert all allArrays from values into one DataFrame 
#and we will add two columns to this final DataFrame:'Bird species' and 'Filepath'
df_list = [] #create an empty list
comb_df = pd.DataFrame(df_list) #created empty list to convert it to an empty DataFrame

for value in values:
    df = pd.DataFrame(value[0], columns=["height", "width", "red","green","blue"])  #value[0] = allArray
    df.loc[df.index[:], 'Bird species'] = value[1]    #value[1] = species name
    df.loc[df.index[:], 'Filepath'] = value[2]   #value[2] = filepath
    comb_df = (pd.concat([comb_df, df], axis=0)) #combine current df with df from loop
    
test_csv_repo_dir = change(r'C:\Users\elanu\Desktop\bird_rep\test_image_bird.csv') 
# test_csv_repo_dir = 
comb_df.to_csv(test_csv_repo_dir)
# df = pd.read_csv(test_csv_repo_dir)
# df[:360000]

,Unnamed: 0,height,width,red,green,blue,Bird species,Filepath
0,0,0,0,38,7,5,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg
1,1,0,1,28,0,0,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg
2,2,0,2,28,4,0,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg
3,3,0,3,27,6,1,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg
4,4,0,4,35,14,9,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg
...,...,...,...,...,...,...,...,...
50171,50171,223,219,42,63,6,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg
50172,50172,223,220,36,60,2,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg
50173,50173,223,221,33,57,0,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg
50174,50174,223,222,34,59,1,ABBOTTS BABBLER,/test/ABBOTTS BABBLER/1.jpg


In [53]:
values = []
c = 0

for row in train_list:
#     if c == 2:
#         break
    pic_path = folder_train.replace('/train', row[0])
    colourImg = Image.open(pic_path)
    colourPixels = colourImg.convert("RGB")
    colourArray = np.array(colourPixels.getdata()).reshape(colourImg.size + (3,))
    indicesArray = np.moveaxis(np.indices(colourImg.size), 0, 2) #changes (2, 256, 256)(=colourImg.size) to (256, 256, 2)
    allArray = np.dstack((indicesArray, colourArray)).reshape((-1, 5))
    value = [] #[array, 'folder/insectname' , '/folder/img.jpg']
    value.append(allArray)
    value.append(row[1])
    value.append(row[0])
    values.append(value)
#     c += 1


df_list = [] #create an empty list
comb_df = pd.DataFrame(df_list) #created empty list to convert it to an empty DataFrame

for value in values:
    df = pd.DataFrame(value[0], columns=["height", "width", "red","green","blue"])  #value[0] = allArray
    df.loc[df.index[:], 'Bird species'] = value[1]    #value[1] = folder/insect_name
    df.loc[df.index[:], 'Filepath'] = value[2]   #value[2] = filepath
    comb_df = (pd.concat([comb_df, df], axis=0)) #combine current df with df from loop
    
train_csv_repo_dir = change(r'C:\Users\elanu\Desktop\bird_rep\train_image_bird.csv') 
# train_csv_repo_dir = 
comb_df.to_csv(train_csv_repo_dir)
# df = pd.read_csv(train_csv_repo_dir)
# df[:360000]

,Unnamed: 0,height,width,red,green,blue,Bird species,Filepath
0,0,0,0,180,172,149,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/001.jpg
1,1,0,1,186,178,155,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/001.jpg
2,2,0,2,187,179,156,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/001.jpg
3,3,0,3,192,186,164,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/001.jpg
4,4,0,4,198,192,170,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/001.jpg
...,...,...,...,...,...,...,...,...
100347,50171,223,219,59,86,77,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/002.jpg
100348,50172,223,220,60,87,78,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/002.jpg
100349,50173,223,221,60,87,78,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/002.jpg
100350,50174,223,222,60,86,77,ABBOTTS BABBLER,/train/ABBOTTS BABBLER/002.jpg


In [54]:
values = []
c = 0

for row in valid_list:
#     if c == 3:
#         break
    pic_path = folder_valid.replace('/valid', row[0])
    colourImg = Image.open(pic_path)
    colourPixels = colourImg.convert("RGB")
    colourArray = np.array(colourPixels.getdata()).reshape(colourImg.size + (3,))
    indicesArray = np.moveaxis(np.indices(colourImg.size), 0, 2) #changes (2, 256, 256)(=colourImg.size) to (256, 256, 2)
    allArray = np.dstack((indicesArray, colourArray)).reshape((-1, 5))
    value = [] #[array, 'folder/insectname' , '/folder/img.jpg']
    value.append(allArray)
    value.append(row[1])
    value.append(row[0])
    values.append(value)
#     c += 1


df_list = [] #create an empty list
comb_df = pd.DataFrame(df_list) #created empty list to convert it to an empty DataFrame

for value in values:
    df = pd.DataFrame(value[0], columns=["height", "width", "red","green","blue"])  #value[0] = allArray
    df.loc[df.index[:], 'Bird species'] = value[1]    #value[1] = folder/insect_name
    df.loc[df.index[:], 'Filepath'] = value[2]   #value[2] = filepath
    comb_df = (pd.concat([comb_df, df], axis=0)) #combine current df with df from loop
    
valid_csv_repo_dir = change(r'C:\Users\elanu\Desktop\bird_rep\valid_image_bird.csv') 
# valid_csv_repo_dir = 
comb_df.to_csv(valid_csv_repo_dir)
# df = pd.read_csv(valid_csv_repo_dir)
# df[:360000]

,Unnamed: 0,height,width,red,green,blue,Bird species,Filepath
0,0,0,0,38,7,5,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/1.jpg
1,1,0,1,28,0,0,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/1.jpg
2,2,0,2,28,4,0,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/1.jpg
3,3,0,3,27,6,1,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/1.jpg
4,4,0,4,35,14,9,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/1.jpg
...,...,...,...,...,...,...,...,...
150523,50171,223,219,102,111,66,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/3.jpg
150524,50172,223,220,100,109,64,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/3.jpg
150525,50173,223,221,102,111,66,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/3.jpg
150526,50174,223,222,102,111,66,ABBOTTS BABBLER,/valid/ABBOTTS BABBLER/3.jpg
